# Integrators
- numerical procedure for solving ordinary differential equations (ODEs) with a given initial value

In [ ]:
%matplotlib notebook
import numpy as np
from numba import jit
from potentials import *
from distances import *
from sampling import *
from integrators import *
from optimize import *
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D

### potential

In [ ]:
def pot_grad(position, boxsize = (0., 1.), sig = 1., eps = 1., pbc = False):
    return gradients.harmonic(position, boxsize, r0=0.5)
def pot(position, boxsize = (0., 1.), sig = 1., eps = 1., pbc = False):
    return potentials.harmonic(position, boxsize, r0=0.5)

### sampling

In [ ]:
position_mcmc = mcmc(potentials.LJ, 3, 2, 100, 0.01, save_config=True)[-1]
position_init = descent(position_mcmc*5, pot_grad, save_config=True)[0][-1]

- velocity
- one particle coordination

In [ ]:
#position_init = np.array([1., 1.])
velocity_init = np.zeros_like(position_init)
velocity_init[0] = [0.5, 1.]
#position_init[0] = [10., 3.]
#print(position_init)
#print(distances(vectors(position_init, boxsize=(0,1))))

### Euler algorithm
- is a first-order scheme
- approximating the integral with the finite sum 
- by the Taylor expansion, the quadratic and higher-order terms are ignored

Approximation 
$$ x(t+\tau)\approx x(t)+\tau\, v(t)\\
v(t+\tau)\approx v(t)+\tau\, a(t)=v(t)-\frac{\tau}{m}\,\nabla\phi(x(t)) \\
\text{with}\,\, v= \frac{\partial x}{\partial t}; \,\, \text{and}\,\, a=\frac{F}{m}=-\frac{1}{m}\nabla \phi(x(t))$$

In [ ]:
position_matrixe, velocity_matrixe, acceleration_matrixe = euler(pot_grad, position_init, velocity_init, 1, 100, 0.001)

### Velocity-Verlet
- approximating the integral with the finite sum by the Taylor expansion
- the cubic and higher-order terms are ignored
Approximation

$$
x(t+\tau)\approx x(t)+\tau \,v(t)-\frac{\tau ^2}{2m}\nabla\phi(x(t))\\
v(t+\tau)\approx v(t)-\frac{\tau }{2m}\left(\nabla\phi(x(t))+\nabla\phi(x(t+\tau))\right)
$$


In [ ]:
position_matrixv, velocity_matrixv, acceleration_matrixv = vv(pot_grad, position_init, velocity_init, 1, 100, 0.001)

### Langevin
- integration schemes that include a thermostat
- full algorithms to simulate molecular dynamics in the N V T ensemble
- stochastic dynamics, based on Langevin dynamics
$$
dp = M^{-1}v \,\text{d}t\\
dv = -\nabla \phi(p)\,\text{d}t- \gamma v \,\text{d}t + \sigma M^{1/2}\,\text{d}W
$$ 
     - the first part of the equation is equal to Newtonian dynamics
     - and the function of the last two terms is to act as a thermostat (friction + noise)

whereby:
- $W = W (t)$ is a vector of 3N independent Wiener processes ? => results in the matrix of noise intensities and a vector of uncorrelated Gaussian random numbers $R_t$, 
* $\gamma > 0$ is a free (scalar) parameter the isotopic friction constant which couple the system to the bath (damping parameter), 
* choosing $ \sigma = \sqrt{2\gamma \beta ^{-1}}$ it is possible to show that the unique probability distribution sampled by the dynamics is the canonical (Gibbs-Boltzmann) density

integration by discretizing in time using a second-order numerical method  according to 
$$\hat{L}*= L*_{\text{LD}}+ \delta t ^2 L*_2 + O(\delta t^3)$$
instead of Taylor series expansion

for the BAOAB method the Langevin dynamics are breaked into three pieces
$$
\left[ \begin{matrix}dp\\dv\end{matrix}\right]= \left[ \begin{matrix}M^{-1}v\\0\end{matrix}\right]\text{d}t+\left[ \begin{matrix}0\\-\nabla\phi(p)\end{matrix}\right]\text{d}t
+\left[ \begin{matrix}0\\-\gamma v \text{d}t + \sigma M^{1/2}\text{d}W\end{matrix}\right]$$

- firts part is labelled A, second as B and third O
O piece is associated with an Ornstein-Uhlenbeck equation with “exact” solution:
$$v(t) = e^{-\gamma t} v(0)+ \frac{\sigma}{\sqrt{2\gamma}}\sqrt{1-e^{-2\gamma t}}M^{1/2}R_t$$
where $R_t$ is a vector of uncorrelated noise processes

- the sequence is given through BAOAB
- to ensure the method is symmetric  all “A” and “B” parts in BAOAB are integrated for a half timestep 



In [ ]:
position_matrixl, velocity_matrixl, acceleration_matrixl = langevin(pot_grad, position_init, velocity_init, 1, 100, 0.001)

### phase volume preservation

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 6))
for ax, integrator in zip(axes.flat, (euler, vv, langevin)):
    x, v, _ = integrator(
        pot_grad, position_init, velocity_init, 1, 100, 0.1)
    ax.plot(x[:,0], v[:,0])
    ax.set_aspect('equal')
    ax.set_xlabel(r'$p$')
    ax.set_ylabel(r'$v$')
fig.tight_layout()

#### Euler
- explicit Euler 
- do not perserve energy over time or volume in phase space
    - trajectories diverge
- deckt sich mit stability analysis des euler shemes 

#### Velocity-Verlet
- Reversible integrator 
- symplectic
    - conservation of volume occupied in phase space
    
#### Langevin
- no damping $\gamma = 0$ $\rightarrow$ Newton dynamics 
    - symplectic
        - conservation of volume occupied in phase space

## pbc vs. box
1. particle in center of box, HO at box edges, pbc
    - code pbc
2. particle in center of box, HO at box edges, barrier potential
    - code barrier potential

In [ ]:
def pot_grad(position, boxsize = (0., 1.), sig = 1., eps = 1., pbc = True):
    #vecs = vectors(position, boxsize, pbc = True)
    return gradients.harmonic(position, boxsize, r0=1)

position_init = np.array([0.5, 0.5])
velocity_init = np.zeros_like(position_init)

position_matrix, velocity_matrix, acceleration_matrix = vv(pot_grad, position_init, velocity_init, 1, 10, 0.001, (0, 1), pbc=True)


In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
r_init = position_matrix[0]
r_matrix = position_matrix
v_matrix = velocity_matrix
colors = np.arange(len(r_init))
scat = ax.scatter(r_init[:,0], r_init[:,1], c=colors)
ax.set_xlabel(r'$p_x$')
ax.set_ylabel(r'$p_y$')
circles = [plt.Circle(r, radius=0.2, fill=False) for i,r in enumerate(r_init)]
for c in circles:
    plt.gca().add_patch(c)
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=50, width=0.005)
ax.set_xlim(-0.2, 1.2)
ax.set_ylim(-0.2, 1.2)
ax.plot(position_matrix[:,0, 0], position_matrix[:,0,1])
def animate(i):
    index = 4*i
    data = r_matrix[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=1, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)


In [ ]:
def pot_grad(position, boxsize = (0., 1.), sig = 1., eps = 1., pbc = False):
    #vecs = vectors(position, boxsize, pbc = True)
    return gradients.harmonic(position, boxsize, r0=1)+ gradients.pot_barrier(position, boxsize)

position_init = np.array([0.5, 0.5])
velocity_init = np.zeros_like(position_init)

position_matrix, velocity_matrix, acceleration_matrix = vv(pot_grad, position_init, velocity_init, 1, 10, 0.001, (0, 1), pbc=False)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
r_init = position_matrix[0]
r_matrix = position_matrix
v_matrix = velocity_matrix
colors = np.arange(len(r_init))
scat = ax.scatter(r_init[:,0], r_init[:,1], c=colors)
ax.set_xlabel(r'$p_x$')
ax.set_ylabel(r'$p_y$')
circles = [plt.Circle(r, radius=0.2, fill=False) for i,r in enumerate(r_init)]
for c in circles:
    plt.gca().add_patch(c)
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=50, width=0.005)
ax.set_xlim(-0.2, 1.2)
ax.set_ylim(-0.2, 1.2)
ax.plot(position_matrix[:,0, 0], position_matrix[:,0,1])
def animate(i):
    index = 4*i
    data = r_matrix[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=1, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

##  visualisation trajectories
1. one paricle with initial velocity, LJ, barrier potential
2. one paricle with initial velocity, LJ, pbc

In [ ]:
def pot_grad_b(position, pbc = False, boxsize = (-5., 5.), sig = 1, eps = 1):
    vecs = vectors(position, boxsize, pbc = False)
    return gradients.LJ(vecs, eps, sig) + gradients.pot_barrier(position, boxsize)
def pot_b(position, sig = 1, eps = 1, boxsize = (-5, 5), pbc = False):
    vecs = vectors(position, boxsize, pbc)
    dist = distances(vecs)
    return potentials.LJ(dist, eps, sig) + potentials.pot_barrier(position, boxsize)

position_mcmc_b = mcmc(pot_b, 100, 2, 10, 0.001, boxsize = (-5, 5))

In [ ]:
position_init_b = descent(position_mcmc_b, pot_grad_b, boxsize=(-5,5), save_config=False)

In [ ]:
velocity_init_b = np.zeros_like(position_init_b)
position_init_b[0] = [10., 0.]
velocity_init_b[0] = [-3., 0.]

In [ ]:
position_matrix_b, velocity_matrix_b, acceleration_matrix_b = vv(pot_grad_b, position_init_b, velocity_init_b, 1, 10, 1e-3, (-5, 5))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
r_init = position_matrix_b[0]
r_matrix = position_matrix_b
v_matrix = velocity_matrix_b
colors = np.arange(len(r_init))
scat = ax.scatter(r_init[:,0], r_init[:,1], c=colors)
ax.set_xlabel(r'$p_x$')
ax.set_ylabel(r'$p_y$')
circles = [plt.Circle(r, radius=0.5, fill=False) for i,r in enumerate(r_init)]
for c in circles:
    plt.gca().add_patch(c)
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=50, width=0.005)
ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.plot(position_matrix_b[:,0, 0], position_matrix_b[:,0,1])
def animate(i):
    index = 4*i
    data = r_matrix[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=1, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

In [ ]:
position = position_matrix_b[-1]
dist=distances(vectors(position, boxsize=(-5,5), pbc=False))
dim, boxsize = 2, (1,2)
dmax = np.sqrt(dim)*(boxsize[1]-boxsize[0])
plt.figure()
def raddisfunc(dist, boxsize=(-5,5), dim=2, sigma=1):
    N = len(dist)
    dmax = np.sqrt(dim)*(boxsize[1]-boxsize[0])
    dmin = 0
    edges = np.linspace(dmin, dmax, 21)
    ndist = int(N-1)
    d = dist[0][1:]
    rd = np.array([np.sum(np.logical_and((d >= emin).astype(int), (d < emax).astype(int)))/ndist for emin, emax in zip(edges[:-1], edges[1:])])
    x = (edges[:-1] + edges[1:])/2
    return x, rd


res = raddisfunc(distances(vectors(position, boxsize=(-5,5), pbc=False)), dim=position.shape[-1], sigma=1)
plt.plot(res[0], res[1])

In [ ]:
def pot_grad_p(position, boxsize = (-5., 5.), pbc=True, sig = 1, eps = 1):
    vecs = vectors(position, boxsize, pbc = True)
    return gradients.LJ(vecs, eps, sig)
def pot_p(position, sig = 1, eps = 1, boxsize = (-5, 5), pbc = True):
    vecs = vectors(position, boxsize, pbc)
    dist = distances(vecs)
    return potentials.LJ(dist, eps, sig) 

position_mcmc_p = mcmc(pot_p, 10, 2, 10, 0.001, boxsize = (-5,5), pbc=True)

In [ ]:
position_init_p = descent(position_mcmc_p, pot_grad_p, boxsize=(-5,5), save_config=False)

In [ ]:
velocity_init_p = np.zeros_like(position_init_p)
position_init_p[0] = [7., 0.]
velocity_init_p[0] = [-1., 0.]

In [ ]:
position_matrix_p, velocity_matrix_p, acceleration_matrix_p = vv(pot_grad_p, position_init_p, velocity_init_p, 1, 10, 1e-4, (-5, 5), pbc=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
r_init = position_matrix_p[0]
r_matrix = position_matrix_p
v_matrix = velocity_matrix_p
colors = np.arange(len(r_init))
scat = ax.scatter(r_init[:,0], r_init[:,1], c=colors)
ax.set_xlabel(r'$p_x$')
ax.set_ylabel(r'$p_y$')
circles = [plt.Circle(r, radius=0.5, fill=False) for i,r in enumerate(r_init)]
for c in circles:
    plt.gca().add_patch(c)
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=50, width=0.005)
ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.plot(position_matrix_p[:,0, 0], position_matrix_p[:,0,1])
def animate(i):
    index = 4*i
    data = r_matrix[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=1, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

In [ ]:
position = position_matrix_p[-1]
dist=distances(vectors(position, boxsize=(-5,5), pbc=True))
dim, boxsize = 2, (1,2)
dmax = np.sqrt(dim)*(boxsize[1]-boxsize[0])
plt.figure()
def raddisfunc(dist, boxsize=(-5,5), dim=2, sigma=1):
    N = len(dist)
    dmax = np.sqrt(dim)*(boxsize[1]-boxsize[0])
    dmin = 0
    edges = np.linspace(dmin, dmax, 21)
    ndist = int(N-1)
    d = dist[0][1:]
    rd = np.array([np.sum(np.logical_and((d >= emin).astype(int), (d < emax).astype(int)))/ndist for emin, emax in zip(edges[:-1], edges[1:])])
    x = (edges[:-1] + edges[1:])/2
    return x, rd


res = raddisfunc(distances(vectors(position, boxsize=(-5,5), pbc=True)), dim=position.shape[-1], sigma=1)
plt.plot(res[0], res[1])

### Langevin

In [ ]:
def pot_grad(position, pbc = False, boxsize = (-5., 5.), sig = 1, eps = 1):
    vecs = vectors(position, boxsize, pbc = False)
    return gradients.LJ(vecs, eps, sig) + gradients.pot_barrier(position, boxsize)*5
def pot(position, sig = 1, eps = 1, boxsize = (-5, 5), pbc = False):
    vecs = vectors(position, boxsize, pbc)
    dist = distances(vecs)
    return potentials.LJ(dist, eps, sig) + potentials.pot_barrier(position, boxsize)*5

position_mcmc = mcmc(pot, 5, 2, 10, 0.001, boxsize = (-5, 5))

In [ ]:
position_init = descent(position_mcmc, pot_grad, boxsize=(-5,5), save_config=False)

In [ ]:
velocity_init = np.zeros_like(position_init)
#position_init_b[0] = [10., 0.]
#velocity_init_b[0] = [-3., 0.]

In [ ]:
position_matrix, velocity_matrix, acceleration_matrix = langevin(pot_grad, position_init, velocity_init, 1, 10, 1e-3, 0.1, 1, 0, (-5, 5))

In [ ]:
plt.figure()
plt.plot(position_matrix[:,0, 0], velocity_matrix[:,0,0])
plt.xlabel(r'$p$')
plt.ylabel(r'$v$')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
r_init = position_matrix[0]
r_matrix = position_matrix
v_matrix = velocity_matrix
colors = np.arange(len(r_init))
scat = ax.scatter(r_init[:,0], r_init[:,1], c=colors)
ax.set_xlabel(r'$p_x$')
ax.set_ylabel(r'$p_y$')
circles = [plt.Circle(r, radius=0.5, fill=False) for i,r in enumerate(r_init)]
for c in circles:
    plt.gca().add_patch(c)
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=50, width=0.005)
ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.plot(position_matrix[:,0, 0], position_matrix[:,0,1])
def animate(i):
    index = 4*i
    data = r_matrix[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=1, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)